In [33]:
import re
import requests
import pandas as pd
from lxml import etree
from datetime import datetime,timedelta
import time
import json
import os

# 认识天天基金网的数据接口

参考：

https://blog.csdn.net/yys1164922014/article/details/100098533?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.channel_param&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.channel_param

## 基金代码和基金名称

这是所有基金代码、基金名称简称的集合http://fund.eastmoney.com/js/fundcode_search.js

数据量是接近9000条数据。这个可以简单的用个正则表达，re.findall("(.*?)",url_text)匹配一下存在电脑上。然后再弄成excel文件。比较简单。

In [16]:
url = 'http://fund.eastmoney.com/js/fundcode_search.js'
len_per_fund = 5 # 每个基金的信息有5个字段

In [17]:
response = requests.get(url)
url_text = response.content.decode('utf-8')
url_text_list = re.findall('\"(.*?)\"',url_text)
num5 = int(len(url_text_list)/5)

In [18]:
url_text_list

['000001',
 'HXCZHH',
 '华夏成长混合',
 '混合型-偏股',
 'HUAXIACHENGZHANGHUNHE',
 '000002',
 'HXCZHH',
 '华夏成长混合(后端)',
 '混合型-偏股',
 'HUAXIACHENGZHANGHUNHE',
 '000003',
 'ZHKZZZQA',
 '中海可转债债券A',
 '债券型-可转债',
 'ZHONGHAIKEZHUANZHAIZHAIQUANA',
 '000004',
 'ZHKZZZQC',
 '中海可转债债券C',
 '债券型-可转债',
 'ZHONGHAIKEZHUANZHAIZHAIQUANC',
 '000005',
 'JSZQXYDQZQ',
 '嘉实增强信用定期债券',
 '债券型-长债',
 'JIASHIZENGQIANGXINYONGDINGQIZHAIQUAN',
 '000006',
 'XBLDLHCZHHA',
 '西部利得量化成长混合A',
 '混合型-偏股',
 'XIBULIDELIANGHUACHENGZHANGHUNHEA',
 '000008',
 'JSZZ500ETFLJA',
 '嘉实中证500ETF联接A',
 '指数型-股票',
 'JIASHIZHONGZHENG500ETFLIANJIEA',
 '000009',
 'YFDTTLCHBA',
 '易方达天天理财货币A',
 '货币型',
 'YIFANGDATIANTIANLICAIHUOBIA',
 '000010',
 'YFDTTLCHBB',
 '易方达天天理财货币B',
 '货币型',
 'YIFANGDATIANTIANLICAIHUOBIB',
 '000011',
 'HXDPJXHHA',
 '华夏大盘精选混合A',
 '混合型-偏股',
 'HUAXIADAPANJINGXUANHUNHEA',
 '000012',
 'HXDPJXHHA',
 '华夏大盘精选混合A(后端)',
 '混合型-偏股',
 'HUAXIADAPANJINGXUANHUNHEA',
 '000013',
 'YFDTTLCHBR',
 '易方达天天理财货币R',
 '货币型',
 'YIFANGDATIANTIANLICAIHUOBIR',
 '000014

In [22]:
# dict(fund_id:fund_name)
w2dict = {}
for i in range(num5):
    tmp = url_text_list[i*len_per_fund:(i+1)*len_per_fund]
    w2dict[tmp[0]] = dict(
                        suoxie=tmp[1],
                        name=tmp[2], 
                        style=tmp[3],
                        pinyin=tmp[4]
                        )
fjson = json.dumps(w2dict)
f = open('fund_info.json','w')
f.write(fjson)
f.close()

In [6]:
with open("基金列表.txt","w") as f:
    
    wlines = []
    for i in range(num5):
        wline = '\t'.join(url_text_list[i*len_per_fund:(i+1)*len_per_fund])
        wlines.append(wline)
    wlines = '\n'.join(wlines)
    f.write(wlines)

In [21]:
dict(fds=121,)

{'fds': 121}

## 基金历史净值接口

url=‘http://fund.eastmoney.com/f10/F10DataApi.aspx?type=lsjz&code=110022&page=10&sdate=2019-01-01&edate=2019-02-13&per=1’
参数情况一目了然。

url变量：

per=1值代表每页的数据量。最多可以有二十行，per=20。设置更高也没用。

code：基金代码

page：当前是第几页，从1开始到pages为止

sdate：开始日期，在最后一页

edate：结束日期，在第一页

显示变量：

records: 在当前条件下，一共有多少条记录

pages：一共有多少页

curpage:当前页码，与url中的page相同


为了代码更加简洁，我设置的每页数据量是1条。这样就更辛苦电脑了。
由于我们这要爬取全部的历史净值，所以要查询到每只基金的起始日期。

var apidata={ content:"
净值日期	单位净值	累计净值	日增长率	申购状态	赎回状态	分红送配

2021-07-19	1.0295	1.0295	-0.38%	场内买入	场内卖出	
2021-07-16	1.0334	1.0334	-0.03%	场内买入	场内卖出	
",records:414,pages:207,curpage:21};

## 查询基金起始日期

http://fund.eastmoney.com/000001.html?spm=search

自己更改中间的基金代码就好
问题是并不是所有基金都能查到起始日期。所以在用re.findall匹配到起止日期的时候，我又加了个IF语句。以防读取不到，产生错误。

In [2]:
def get_start_date(fund_id):
    # 返回日期字符串
    search_url='http://fund.eastmoney.com/'+fund_id+'.html?spm=search'
    # 获取网页内容；设置网页编码，获取起始日期
    response = requests.get(search_url)
    text = response.content.decode('utf-8')
    
    start_date = re.findall('<td><span class="letterSpace01">成 立 日</span>：(.*?)</td>', text)
    # 对无记录的基金代码进行记录，方便后续清除
    if start_date == []:
        start_date = ['无记录'] 
    return start_date[0] # 如果信息正常就会返回开始日期，如果有错误，就会返回无记录。都是字符串的结果

## 根据基金的每一条数据接口的网址，把那条信息抓下来，返回list结果

一条网址的信息是这样：

var apidata={ content:"
净值日期	单位净值	累计净值	日增长率	申购状态	赎回状态	分红送配
2021-07-16	1.0334	1.0334	-0.03%	场内买入	场内卖出	
",records:414,pages:207,curpage:21};

In [87]:
response = requests.get('https://fundf10.eastmoney.com/F10DataApi.aspx?type=lsjz&code=512000&page=21&sdate=2020-01-01&edate=2021-09-13&per=1')
text = response.text
print(text)
text_html = re.findall(',records:(.*?),pages:',text)
#html = etree.HTML(text_html)
print(text_html)



var apidata={ content:"<table class='w782 comm lsjz'><thead><tr><th class='first'>净值日期</th><th>单位净值</th><th>累计净值</th><th>日增长率</th><th>申购状态</th><th>赎回状态</th><th class='tor last'>分红送配</th></tr></thead><tbody><tr><td>2021-08-16</td><td class='tor bold'>1.0422</td><td class='tor bold'>1.0422</td><td class='tor bold red'>0.80%</td><td>场内买入</td><td>场内卖出</td><td class='red unbold'></td></tr></tbody></table>",records:414,pages:414,curpage:21};
['414']


In [29]:
def make_fund_json(fund_info_url='http://fund.eastmoney.com/js/fundcode_search.js',save=False):
    url = fund_info_url
    len_per_fund = 5 # 每个基金的信息有5个字段
    
    response = requests.get(url)
    url_text = response.content.decode('utf-8')
    url_text_list = re.findall('\"(.*?)\"',url_text)
    num5 = int(len(url_text_list)/len_per_fund)
    
    # dict(fund_id:fund_name)
    w2dict = {}
    for i in range(num5):
        tmp = url_text_list[i*len_per_fund:(i+1)*len_per_fund]
        w2dict[tmp[0]] = dict(
                            suoxie=tmp[1],
                            name=tmp[2], 
                            style=tmp[3],
                            pinyin=tmp[4]
                            )
    fjson = json.dumps(w2dict)
    if save:
        f = open('fund_info.json','w')
        f.write(fjson)
        f.close()
    return w2dict

In [25]:
def get_fund_json(fund_json_path='fund_info.json'):
    f = open('fund_info.json','r')
    fjson = f.read()
    fund_dict = json.loads(fjson)
    f.close()
    return fund_dict

In [3]:
def get_fund_info_per1_list(url_fund):
    # 每个基金一页一天数据，per=1，返回list
    
    list_info = [] # 存储基金信息
    
    # re.findall,xpath 匹配信息
    response = requests.get(url_fund)
    text = response.text
    text_html = re.findall('content:"(.*?)",records:',text)[0]
    html = etree.HTML(text_html)
    html_data = html.xpath('//tr/td')
    for info in html_data:
        list_info.append(info.text)  # 每个字段一行：净值日期 单位净值 累计净值 日增长率 申购状态 赎回状态 分红送配
    
    return list_info

In [4]:
def get_timestamp(date_str):
    # 输入'2012-02-19' 得到浮点数时间戳
    return time.mktime(time.strptime(date_str,"%Y-%m-%d"))

In [5]:
def get_cnt_days(sdate,edate):  # 计算时间段间天数，截止时间是否大于起止时间
    stime = time.mktime(time.strptime(sdate,"%Y-%m-%d"))
    etime = time.mktime(time.strptime(edate,"%Y-%m-%d"))
    cnt_days = count_days = int((edate - sdate)/(24*60*60))+1
    return (cnt_days, False if cnt_days<0 else True)

In [6]:
def get_fund_data_url(fund_id='512000',page=1,sdate='2021-09-16',edate='2021-09-16',per=1):
    return 'https://fundf10.eastmoney.com/F10DataApi.aspx?type=lsjz&code={}&page={}&sdate={}&edate={}&per={}'.format(fund_id,page,sdate,edate,per)

In [40]:
def get_fund_data_list(fund_id,pages,start_date,end_date,per):
    fund_data_list = []
    for page in range(pages):
        fund_id_url = get_fund_data_url(fund_id,page+1,start_date,end_date,per)
        #print("fund_id_url:{}".format(fund_id_url))
        tmp_list = get_fund_info_per1_list(fund_id_url)
        if tmp_list == ['暂无数据']:
            break
        else:
            fund_data_list.append(tmp_list)
        #print("current data:{}".format(tmp_list))
    return fund_data_list

In [8]:
def get_cnt_pages(tmp_fund_data_url):
    # 输入基金数据网址，反悔此条件下一共有多少页数据
    response = requests.get(tmp_fund_data_url)
    text = response.text
    #print(text)
    text_html = re.findall(',records:(.*?),pages:',text)
    #print(text_html)
    return int(text_html[0])

In [37]:
def save2excel(fund_id_list,results,file_path='../fund_res.xlsx',use_name=False):
    writer = pd.ExcelWriter(file_path)
    if use_name:
        if os.path.exists('fund_info.json'):
            fund_json = get_fund_json()
        else:
            fund_json = make_fund_json()
    for idx,fund_id in enumerate(fund_id_list):
        df_fund = pd.DataFrame(results[idx][::-1])
        df_fund.columns=['净值日期','单位净值','累计净值','日增长率','申购状态','赎回状态','分红送配']
        if use_name:
            df_fund.to_excel(writer,sheet_name=str(fund_id+fund_json[fund_id]['name']))
        else:
            df_fund.to_excel(writer,sheet_name=str(fund_id))
        print("Finished {}".format(fund_id))
    writer.save()
    print("Finished [{}] .".format(file_path))

In [55]:
#fund_id_list = ['512000','512880','512070']  # 基金代码
#fund_id_list = ['513050','513130','159920','159740']
fund_id_list = ['159938','512980'] 
# 每个基金的时间区间
time_interval = [['2016-08-30','2021-09-16'],
                 ['2016-08-30','2021-09-16'],
                 ['2016-08-30','2021-09-16'],
                 ['2014-01-01','2021-09-16'],
                 ['2014-01-01','2021-09-16'],
                 ['2014-01-01','2021-09-16'],
                ]
#fund_id = 512000
#start_date = '2021-09-13'
#end_date = '2021-09-16'
per = 1

In [56]:
get_results = []
for idx,fund_id in enumerate(fund_id_list):
    
    # 获取时间节点并检查
    # todo：检查结束时间
    start_date,end_date = time_interval[idx]  # 获取时间区间
    build_date = get_start_date(fund_id)
    build_time_stamp = get_timestamp(build_date)
    start_time_stamp = get_timestamp(start_date)
    if start_time_stamp < build_time_stamp:
        start_date = build_date  # 最早开始时间是建立时间
        print("{}的最早开始时间更正为创建时间:{}".format(fund_id,start_date))
    
    # 获取当前条件下一共多少页数据
    pages = get_cnt_pages(get_fund_data_url(fund_id,1,start_date,end_date,per))
    
    print("fund_id:{},[{}-{}],per:{},pages:{}".format(fund_id,start_date,end_date,per,pages))
    # 提取基金数据
    fund_data = get_fund_data_list(fund_id,pages,start_date,end_date,per)
    get_results.append(fund_data) # 此时每个基金的结果list为倒排需要用get_results[?][::-1]

fund_id:159938,[2016-08-30-2021-09-16],per:1,pages:1235
512980的最早开始时间更正为创建时间:2017-12-27
fund_id:512980,[2017-12-27-2021-09-16],per:1,pages:887


In [57]:
save2excel(fund_id_list,get_results,file_path='../fund_res_YC.xlsx',use_name=True)

Finished 159938
Finished 512980
Finished [../fund_res_YC.xlsx] .


In [36]:
df_found_i = pd.DataFrame(get_results[0])
########为DataFrame数据设置columns
df_found_i.columns=['净值日期','单位净值','累计净值','日增长率','申购状态','赎回状态','分红送配']
#########为了数据便于自己使用，文件命名，依据基金的中文名称 。
df_found_i.to_excel('../%s.xlsx'%(512000))


In [54]:
get_results[0][::-1]

[['2016-08-30', '1.0000', '1.0000', None, '场内买入', '场内卖出', None],
 ['2016-08-31', '1.0011', '1.0011', '0.11%', '场内买入', '场内卖出', None],
 ['2016-09-01', '0.9988', '0.9988', '-0.23%', '场内买入', '场内卖出', None],
 ['2016-09-02', '1.0000', '1.0000', '0.12%', '场内买入', '场内卖出', None],
 ['2016-09-07', '1.0005', '1.0005', None, '场内买入', '场内卖出', None],
 ['2016-09-09', '0.9979', '0.9979', None, '场内买入', '场内卖出', None],
 ['2016-09-13', '0.9893', '0.9893', None, '场内买入', '场内卖出', None],
 ['2016-09-14', '0.9804', '0.9804', '-0.90%', '场内买入', '场内卖出', None],
 ['2016-09-19', '0.9839', '0.9839', '0.36%', '场内买入', '场内卖出', None],
 ['2016-09-20', '0.9822', '0.9822', '-0.17%', '场内买入', '场内卖出', None],
 ['2016-09-21', '0.9889', '0.9889', '0.68%', '场内买入', '场内卖出', None],
 ['2016-09-22', '0.9897', '0.9897', '0.08%', '场内买入', '场内卖出', None],
 ['2016-09-23', '0.9836', '0.9836', '-0.62%', '场内买入', '场内卖出', None],
 ['2016-09-26', '0.9671', '0.9671', '-1.68%', '场内买入', '场内卖出', None],
 ['2016-09-27', '0.9767', '0.9767', '0.99%', '场内买入', '场